In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import joblib

import warnings
warnings.filterwarnings('ignore')

differences 

In [32]:
model_df = pd.read_csv('./data_frames/game_stats_differences_2020.csv')

In [33]:
model_df.head()

,goal_diff,won,game_ids,goals,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,ice_tilt
0,3,yes,2020020001,3,-7,2,0,33.4,1.0,0.0,16.0,-2,0,8,-1058.0
1,4,yes,2020020002,4,10,5,-2,16.7,1.0,1.0,1.8,0,0,2,89.0
2,1,yes,2020020003,1,2,-5,-2,-16.7,0.0,1.0,8.0,-1,-4,-18,1328.0
3,-2,no,2020020004,-2,-4,3,-4,0.0,0.0,2.0,3.6,-1,4,-1,695.0
4,-3,no,2020020005,-3,-5,-8,-4,25.0,1.0,2.0,8.8,3,5,1,405.0


In [34]:
X = model_df.drop(columns=['won', 'goal_diff','goals','game_ids', ])
y = model_df['won']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [5]:
sc = StandardScaler()

Xs_train = sc.fit_transform(X_train)
Xs_test = sc.transform(X_test)

In [6]:
logreg = LogisticRegression(random_state=42, max_iter=1000)

In [7]:
log_params = {
    'C': np.linspace(0.1,4,100)
             }

In [8]:
log_reg = RandomizedSearchCV(logreg,log_params,n_iter=100,n_jobs=-1,verbose=1)

In [9]:
log_reg.fit(Xs_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(estimator=LogisticRegression(max_iter=1000, random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'C': array([0.1       , 0.13939394, 0.17878788, 0.21818182, 0.25757576,
       0.2969697 , 0.33636364, 0.37575758, 0.41515152, 0.45454545,
       0.49393939, 0.53333333, 0.57272727, 0.61212121, 0.65151515,
       0.69090909, 0.73030303, 0.76969697, 0.80909091, 0.84848485,
       0.8878...
       2.66060606, 2.7       , 2.73939394, 2.77878788, 2.81818182,
       2.85757576, 2.8969697 , 2.93636364, 2.97575758, 3.01515152,
       3.05454545, 3.09393939, 3.13333333, 3.17272727, 3.21212121,
       3.25151515, 3.29090909, 3.33030303, 3.36969697, 3.40909091,
       3.44848485, 3.48787879, 3.52727273, 3.56666667, 3.60606061,
       3.64545455, 3.68484848, 3.72424242, 3.76363636, 3.8030303 ,
       3.84242424, 3.88181818, 3.92121212, 3.96060606, 4.        ])},
                   verbose=1)

In [10]:
log_reg.best_params_

{'C': 0.17878787878787877}

In [11]:
log_reg.score(Xs_test,y_test), log_reg.score(Xs_train,y_train)

(0.7419354838709677, 0.6912442396313364)

In [12]:
#joblib.dump(log_reg,'./models/log_reg.pkl')

['./models/log_reg.pkl']

random forest

In [27]:
rfc = RandomForestClassifier()

In [28]:
rfc_params = {
    'n_estimators': list(range(2,51,1)),
    'max_depth': list(range(3,51,1)),
    'min_samples_split': list(range(2,51,1)),
    'min_samples_leaf': list(range(2,51,1)),
    'max_features': ['auto','sqrt','log2']
}

In [29]:
rfc_search = RandomizedSearchCV(rfc,rfc_params,n_iter=1000,n_jobs=-2,verbose=1,random_state=42)

In [35]:
rfc_search.fit(X_train,y_train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


RandomizedSearchCV(estimator=RandomForestClassifier(), n_iter=1000, n_jobs=-2,
                   param_distributions={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10,
                                                      11, 12, 13, 14, 15, 16,
                                                      17, 18, 19, 20, 21, 22,
                                                      23, 24, 25, 26, 27, 28,
                                                      29, 30, 31, 32, ...],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [2, 3, 4, 5, 6, 7,
                                                             8, 9, 10, 11, 12,
                                                             13, 14, 15, 16, 17,
                                                             18, 19, 20, 21, 22,
                                                             23, 24, 25, 

In [36]:
rfc_search.score(X_train,y_train), rfc_search.score(X_test,y_test)

(0.7588325652841782, 0.7235023041474654)

In [ ]:
rfc_search.

In [18]:
rfc_search.best_estimator_.feature_importances_

array([0.04444509, 0.12279573, 0.01589059, 0.31355386, 0.23780591,
       0.01008572, 0.0581746 , 0.0486683 , 0.04723833, 0.0695592 ,
       0.03178267])

In [19]:
rfc_search.best_params_

{'n_estimators': 35,
 'min_samples_split': 12,
 'min_samples_leaf': 17,
 'max_features': 'log2',
 'max_depth': 3}

In [20]:
#joblib.dump(rfc_search,'./models/random_forest_classifier.pkl')

['./models/random_forest_classifier.pkl']

Minute over minute modeling:

differences in stats. 

In [3]:
counter = 200
i = 0
scores_df = pd.DataFrame()
while counter <= 6000:
    df = pd.read_csv(f'./data_frames/{counter}_minutes_diffs_2020.csv')
    X = df.drop(columns='won')
    y = df['won']
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)
    rfc = RandomForestClassifier()
    rfc_params = {
    'n_estimators': list(range(2,51,1)),
    'max_depth': list(range(3,51,1)),
    'min_samples_split': list(range(2,51,1)),
    'min_samples_leaf': list(range(2,51,1)),
    'max_features': ['auto','sqrt','log2']
    }
    rfc_search = RandomizedSearchCV(rfc,rfc_params,n_iter=2000,n_jobs=-2,random_state=42)
    rfc_search.fit(X_train,y_train)
    test_score = rfc_search.score(X_test,y_test)
    train_score = rfc_search.score(X_train,y_train)
    scores = {'test_score': test_score, 'train_score': train_score, 'minutes_into_game':counter}
    temp_df = pd.DataFrame(scores, index=[i])
    scores_df = pd.concat([scores_df, temp_df ], axis=0, ignore_index=True)
    #joblib.dump(rfc_search, f'./models/{counter}_rfc_search_diffs_2020.pkl')
    counter += 200
    i += 1
    
    
    

In [9]:
scores_df.to_csv('./data_frames/minute_rf_scores.csv', index=False)

In [5]:
counter = 200
i = 0
log_scores_df = pd.DataFrame()
while counter <= 6000:
    df = pd.read_csv(f'./data_frames/{counter}_minutes_diffs_2020.csv')
    X = df.drop(columns='won')
    y = df['won']
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)
    sc = StandardScaler()
    Xs_train = sc.fit_transform(X_train)
    Xs_test = sc.transform(X_test)
    logreg = LogisticRegression(max_iter=1000, random_state=42)
    log_params = {'C': np.linspace(0.1,4,100), 'penalty':['l1','l2']}
    log_reg = RandomizedSearchCV(logreg,log_params,n_iter=100,n_jobs=-2, random_state=42)
    log_reg.fit(Xs_train,y_train)
    test_score = log_reg.score(Xs_test,y_test)
    train_score = log_reg.score(Xs_train,y_train)
    scores = {'test_score': test_score, 'train_score': train_score, 'minutes_into_game':counter}
    temp_df = pd.DataFrame(scores, index=[i])
    log_scores_df = pd.concat([log_scores_df, temp_df ], axis=0, ignore_index=True)
    #joblib.dump(rfc_search, f'./models/{counter}_logreg_search_diffs_2020.pkl')
    counter += 200
    i += 1
    

In [8]:
log_scores_df.to_csv('./data_frames/minute_log_reg_scores.csv', index=False)